In [15]:
# !pip install bs4
# !pip install requests
!pip install html5lib

You should consider upgrading via the 'c:\users\dhuma\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [47]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import html5lib

Get the URL in variable

In [48]:
url ="https://www.zipcodesonline.com/2020/06/postal-code-of-toronto-in-2020.html"

Read the HTML using pandas into a dataframe

In [49]:
webdata = pd.read_html(url, flavor ='bs4')
webdata

[                        0
 0  Postal Code of Toronto,
            0                  1             2                 3
 0    SL. NO.      NEIGHBOURHOOD  POSTAL CODES          DISTRICT
 1        NaN                NaN           NaN               NaN
 2          1           Adelaide           M5H  Downtown Toronto
 3          2    Agincourt North           M1V       Scarborough
 4          3          Agincourt           M1S       Scarborough
 ..       ...                ...           ...               ...
 202      201  Woodbine Gardens,           M4B         East York
 203      202   Woodbine Heights           M4C         East York
 204      203    York Mills West           M2P        North York
 205      204         York Mills           M2L        North York
 206      205          Yorkville           M5R   Central Toronto
 
 [207 rows x 4 columns]]

In [322]:
df = pd.DataFrame(webdata[1])
df.head()

,0,1,2,3
0,SL. NO.,NEIGHBOURHOOD,POSTAL CODES,DISTRICT
1,NaN,NaN,NaN,NaN
2,1,Adelaide,M5H,Downtown Toronto
3,2,Agincourt North,M1V,Scarborough
4,3,Agincourt,M1S,Scarborough


Drop SL.NO column

In [323]:
df.drop(df.columns[0],axis=1, inplace =True)
df.head()

,1,2,3
0,NEIGHBOURHOOD,POSTAL CODES,DISTRICT
1,NaN,NaN,NaN
2,Adelaide,M5H,Downtown Toronto
3,Agincourt North,M1V,Scarborough
4,Agincourt,M1S,Scarborough


Add the column names and remove the first row

In [324]:
col = df.iloc[0] 
df.columns = col
df.drop(df.index[0], inplace =True)
df.head()

,NEIGHBOURHOOD,POSTAL CODES,DISTRICT
1,NaN,NaN,NaN
2,Adelaide,M5H,Downtown Toronto
3,Agincourt North,M1V,Scarborough
4,Agincourt,M1S,Scarborough
5,Albion Gardens,M9V,Etobicoke


Rename and re-arrange the columns

In [325]:
df = df.rename(columns= {'NEIGHBOURHOOD':'Neighbourhood','POSTAL CODES':'PostalCode','DISTRICT':'Borough'})
df.head()

,Neighbourhood,PostalCode,Borough
1,NaN,NaN,NaN
2,Adelaide,M5H,Downtown Toronto
3,Agincourt North,M1V,Scarborough
4,Agincourt,M1S,Scarborough
5,Albion Gardens,M9V,Etobicoke


In [326]:
name = list(df.columns[1:]) + [df.columns[0]]
print(name)
df = df[name]
df.head()

['PostalCode', 'Borough', 'Neighbourhood']


,PostalCode,Borough,Neighbourhood
1,NaN,NaN,NaN
2,M5H,Downtown Toronto,Adelaide
3,M1V,Scarborough,Agincourt North
4,M1S,Scarborough,Agincourt
5,M9V,Etobicoke,Albion Gardens


Drop the un-assigned borough

In [327]:
df.dropna(subset=['Borough'],axis=0, inplace = True)
df.head()

,PostalCode,Borough,Neighbourhood
2,M5H,Downtown Toronto,Adelaide
3,M1V,Scarborough,Agincourt North
4,M1S,Scarborough,Agincourt
5,M9V,Etobicoke,Albion Gardens
6,M8W,Etobicoke,Alderwood


In [328]:
df.sort_values(by=['PostalCode'],inplace= True)
df.reset_index(drop = True,inplace= True)
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union
5,M1E,Scarborough,Morningside
6,M1E,Scarborough,West Hill
7,M1E,Scarborough,Guildwood
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


In [431]:
dfNEW=df[['PostalCode','Neighbourhood']]
dfNEW

,PostalCode,Neighbourhood
0,M1B,Rouge
1,M1B,Malvern
2,M1C,Highland Creek
3,M1C,Rouge Hill
4,M1C,Port Union
...,...,...
200,M9V,Mount Olive
201,M9V,Albion Gardens
202,M9V,Humbergate
203,M9W,Northwest


Declare a function to concatenate strings and add , as seperator

In [435]:
dfNEW['Neighbourhood']= dfNEW['Neighbourhood'].apply(lambda x: x +',')
dfNEW.head()

,Neighbourhood
PostalCode,
M1B,"Rouge,Malvern,"
M1C,"Highland Creek,Rouge Hill,Port Union,"
M1E,"Morningside,West Hill,Guildwood,"
M1G,"Woburn,"
M1H,"Cedarbrae,"


In [439]:
dfNEW = dfNEW.groupby(['PostalCode']).sum()
dfNEW['Neighbourhood'] = dfNEW['Neighbourhood'].str.strip(',')
dfNEW.reset_index(inplace = True)
dfNEW.head()

,PostalCode,Neighbourhood
0,M1B,"Rouge,Malvern"
1,M1C,"Highland Creek,Rouge Hill,Port Union"
2,M1E,"Morningside,West Hill,Guildwood"
3,M1G,Woburn
4,M1H,Cedarbrae


Check if any un-assigned Neighbourhood is present in data frame

In [20]:
nuldf = df.notnull()
nuldf['Neighbourhood'].value_counts()

True    205
Name: Neighbourhood, dtype: int64